In [1]:
import os
import csv
from google import genai
from google.genai import types
client = genai.Client(api_key="")
# 指定 txt 文件所在的目录和输出的 csv 文件路径
input_directory = './destination_folder_running'
output_csv_file = './output_final2.csv'
log_file = './processing_errors.log'


def extract_with_model(text_content):
    response = client.models.generate_content(
        model="gemini-2.5-flash-preview-05-20",
        config=types.GenerateContentConfig(system_instruction='''你将分析一篇关于金属有机框架材料（MOFs）对以下五种化合物吸附性能的科技报告：

isoprene
cis-2-pentene
1-pentene
trans-2-pentene
n-pentane

请根据报告内容判断每种化合物是否能被该 MOFs 材料吸附。如果报告中提到了具体的 MOF 材料名称，请将其作为第一个元素，随后依次用 "Can Adsorb" 或 "Cannot Adsorb" 表示对每种化合物的吸附能力。

请严格按照isoprene、cis-2-pentene、1-pentene、trans-2-pentene、n-pentane的顺序。
输出格式应为一个纯文本列表，不包含表头或其他解释性内容，例如：

MOF-123 | Can Adsorb | Cannot Adsorb | Cannot Adsorb | Can Adsorb | Cannot Adsorb

如果你未在报告中找到某种化合物的相关信息，请使用 "Not Mentioned" 代替。''',
        contents=s))
       
    print(response.text.split("|"))
    return response.text.split("|")

def log_error(filename, error_message):
    with open(log_file, 'a', encoding='utf-8') as log:
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        log.write(f'[{timestamp}] Error processing file: {filename}\n')
        log.write(f'Error message: {error_message}\n')
        log.write('-' * 60 + '\n')
        
def read_txt_files(directory):
    all_rows = []

    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    print(f'Processing {filename}...')

                    extracted_row = extract_with_model(content)
                    extracted_row.append(filename)  # 添加文件名到末尾
                    all_rows.append(extracted_row)

            except Exception as e:
                print(f'Failed to process {filename}: {e}')
                log_error(filename, str(e))

    return all_rows

def write_to_csv(data, output_file):
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # 写入表头，根据实际提炼的数据结构调整
        writer.writerow(['MOFs name', 'isoprene', 'cis-2-pentene', '1-pentene', 'trans-2-pentene', 'n-pentane', 'filename'])  # 修改这里以适应你的数据结构
        writer.writerows(data)


if __name__ == '__main__':
    extracted_data = read_txt_files(input_directory)
    write_to_csv(extracted_data, output_csv_file)
    print(f'处理完成，结果已写入 {output_csv_file}')

SyntaxError: '(' was never closed (2913347634.py, line 13)